## Третье ДЗ
### Sentiment Analysis финансовых новостей. Пример: есть текст, мы хотим узнать как он влияет на цену актива

1. **Подготовка данных**
   -  Исходный CSV-файл `news_for_hw.csv` 40к текстов разной длины связанных с золотом:
     - `text` — текст новости
     - `sentiment` — метка (`негативный`, `нейтральный`, `позитивный`)
     - `confidence` — уверенность в метке.
   - разметка текста при помощи https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-mnli-xnli

   - Преобразовали текстовые метки в числовые (`0`, `1`, `2`).

2. **Fine-tuning модели**
   - Взяли предобученную модель **ProsusAI/finbert** из Hugging Face.
   - Сконвертировали `DataFrame` в `datasets.DatasetDict` с разделением на `train`/`test`.
   - Провели токенизацию текстов (`max_length=512`, `padding="max_length"`).
   - Настроили `Trainer` с метриками (`accuracy`, `precision`, `recall`, `f1`) и стратегией оценки по эпохам.

3. **Оценка качества**
   - Получили финальные метрики на тестовом наборе:
      - Epoch	Training Loss	Validation Loss	Accuracy	Precision	Recall	F1
        - 1	0.880800	0.864374	0.601892	0.651298	0.601892	0.567768
        - 2	0.862000	0.863082	0.601425	0.644012	0.601425	0.556303
        - 3	0.826000	0.864676	0.612869	0.678293	0.612869	0.585044

4. **Inference / применение**
   - Создаем `pipeline("sentiment-analysis")`, передаем произвольные тексты и получили предсказанные метки + confidence-скоры.




In [6]:
# # Fine-tune ProsusAI/finbert
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


df = pd.read_csv("news_for_hw.csv")  #  text, sentiment, confidence
label2id = {'негативный':0, 'нейтральный':1, 'позитивный':2}
df['label'] = df['sentiment'].map(label2id)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
def to_hf(ds):
    return Dataset.from_pandas(ds[['text','label']].reset_index(drop=True))

dataset = DatasetDict({
    'train': to_hf(train_df),
    'test':  to_hf(test_df)
})


MODEL = 'ProsusAI/finbert'
tokenizer = AutoTokenizer.from_pretrained(MODEL)

def tokenize_fn(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=512)

dataset = dataset.map(tokenize_fn, batched=True)
dataset = dataset.remove_columns(['text'])
dataset.set_format('torch')


model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': prec,
        'recall': rec,
        'f1': f1
    }

training_args = TrainingArguments(
    output_dir='./out',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    logging_steps=50,
    save_strategy='no',
    no_cuda=not use_cuda,
    fp16=use_cuda
)

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics
)


trainer.train()
metrics = trainer.evaluate()
print("### Eval metrics:", metrics)

preds_output = trainer.predict(dataset['test'])
preds = np.argmax(preds_output.predictions, axis=1)

sample = test_df.reset_index(drop=True).loc[:4, ['text','sentiment']].copy()
sample['predicted'] = [ {0:'негативный',1:'нейтральный',2:'позитивный'}[i] for i in preds[:5] ]
print("\n### Sample predictions:")
print(sample.to_string(index=False))

Map: 100%|██████████| 8563/8563 [00:15<00:00, 541.55 examples/s]


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.880800,0.864374,0.601892,0.651298,0.601892,0.567768
2,0.862000,0.863082,0.601425,0.644012,0.601425,0.556303
3,0.826000,0.864676,0.612869,0.678293,0.612869,0.585044


### Eval metrics: {'eval_loss': 0.8646763563156128, 'eval_accuracy': 0.6128693214994745, 'eval_precision': 0.678292648814983, 'eval_recall': 0.6128693214994745, 'eval_f1': 0.585043615253853, 'eval_runtime': 38.3353, 'eval_samples_per_second': 223.371, 'eval_steps_per_second': 13.982, 'epoch': 3.0}

### Sample predictions:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [8]:
from transformers import pipeline
sent_pipe = pipeline(
    "sentiment-analysis",
    model=trainer.model, 
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1)

my_texts = [
    "Золото не изменилось. Ничего не делай"]

results = sent_pipe(my_texts)
for text, res in zip(my_texts, results):
    print(f"\"{text}\" → {res['label']} (score: {res['score']:.3f})")


Device set to use cuda:0


"Золото не изменилось. Ничего не делай" → neutral (score: 0.617)
